You have to work on the (NYC Trip Fare)[https://www.kaggle.com/api/v1/datasets/download/diishasiing/revenue-for-cab-drivers/archive.zip] repository. You can skip the store_and_fwd_flag column, but it’s a bonus point if you can manage it correctly.

Notes
1. It is mandatory to use GitHub for developing the project.
2. The project must be a jupyter notebook.
3. There is no restriction on the libraries that can be used, nor on the Python version.
4. All questions on the project must be asked in the Discussion forum on the course website.
5. At most 3 students can be in each group. You must create the groups by yourself. You can use the Discussion forum to create the groups.
6. You do not have to send me the project before the discussion.
7. You do not have to prepare any slides for the discussion.

In [ ]:
import pandas as pd
import numpy as np


df = pd.read_csv('data.csv')
# Manage the problem with the store_and_fwd_flag column by replacing NaN values with Unknown so that the column
# doesn't have mixed types.
df["store_and_fwd_flag"].replace({pd.NA: 'Unknown'}, inplace=True)

C:\Users\1\AppData\Local\Temp\ipykernel_20436\3640577101.py:5: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data.csv')


1. Extract all trips with trip_distance larger than 50

In [10]:
df[df["trip_distance"] > 50].head(4)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
23842,2.0,2020-01-01 01:53:07,2020-01-01 03:54:41,1.0,52.30,5.0,N,262,265,1.0,300.0,0.0,0.0,61.78,6.12,0.3,370.70,2.5
39013,2.0,2020-01-01 02:05:07,2020-01-01 03:03:10,1.0,51.23,5.0,N,264,264,1.0,329.0,0.0,0.5,100.78,6.12,0.3,436.70,0.0
41620,1.0,2020-01-01 03:05:54,2020-01-01 04:16:26,1.0,53.80,5.0,N,132,265,1.0,250.0,0.0,0.0,53.35,16.62,0.3,320.27,0.0
58262,2.0,2020-01-01 05:36:12,2020-01-01 06:40:06,1.0,55.23,5.0,N,132,265,2.0,170.0,0.0,0.5,0.00,18.26,0.3,189.06,0.0


2. Extract all trips where payment_type is missing


In [11]:
df[df["payment_type"].isna()].head(4)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
6339567,NaN,2020-01-01 08:51:00,2020-01-01 09:19:00,NaN,13.69,NaN,Unknown,136,232,NaN,51.05,2.75,0.5,0.0,0.0,0.3,54.60,0.0
6339568,NaN,2020-01-01 08:38:43,2020-01-01 08:51:08,NaN,3.42,NaN,Unknown,121,9,NaN,27.06,2.75,0.0,0.0,0.0,0.3,30.11,0.0
6339569,NaN,2020-01-01 08:27:00,2020-01-01 08:32:00,NaN,2.20,NaN,Unknown,197,216,NaN,24.36,2.75,0.5,0.0,0.0,0.3,27.91,0.0
6339570,NaN,2020-01-01 08:46:00,2020-01-01 08:57:00,NaN,0.84,NaN,Unknown,262,236,NaN,26.08,2.75,0.5,0.0,0.0,0.3,29.63,0.0


3. For each (PULocationID, DOLocationID) pair, determine the number of trips

In [23]:
trip_counts = df.groupby(['PULocationID', 'DOLocationID']).size()
trip_counts

PULocationID  DOLocationID
1             1                638
              50                 1
              68                 1
              138                2
              140                1
                              ... 
265           259                2
              261                1
              263                4
              264              317
              265             2508
Length: 31277, dtype: int64

4. Save all rows with missing VendorID, passenger_count, store_and_fwd_flag, payment_type in a new dataframe called bad, and remove those rows from the original dataframe.

5. Add a duration column storing how long each trip has taken (use tpep_pickup_datetime, tpep_dropoff_datetime)

6. For each pickup location, determine how many trips have started there.


7. Cluster the pickup time of the day into 30-minute intervals (e.g. from 02:00 to 02:30)


8. For each interval, determine the average number of passengers and the average fare amount.


9. For each payment type and each interval, determine the average fare amount

10. For each payment type, determine the interval when the average fare amount is maximum


11. For each payment type, determine the interval when the overall ratio between the tip and the fare amounts is maximum

12. Find the location with the highest average fare amount

13. Build a new dataframe (called common) where, for each pickup location we keep all trips to the 5 most common destinations (i.e. each pickup location can have different common destinations).

14. On the common dataframe, for each payment type and each interval, determine the average fare amount

15. Compute the difference of the average fare amount computed in the previous point with those computed at point 9.

16. Compute the ratio between the differences computed in the previous point and those computed in point 9. Note: you have to compute a ratio for each pair (payment type, interval).

17. Build chains of trips. Two trips are consecutive in a chain if (a) they have the same VendorID, (b) the pickup location of the second trip is also the dropoff location of the first trip, (c) the pickup time of the second trip is after the dropoff time of the first trip, and (d) the pickup time of the second trip is at most 2 minutes later than the dropoff time of the first trip.

Hint: Add a column chain to the dataset. A chain can have more than two trips.